## imports

In [333]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import scipy.stats as stats
import os
from os.path import join, exists
import sys
import torch
import torchvision
import torchvision.transforms as transforms
import torchvision.models as models
import torchvision.datasets as datasets
from torch.autograd import Variable as V
from torch.nn import functional as F
import torch.nn  as nn
from PIL import Image
from IPython.core.debugger import set_trace
import nethook as nethook
import scipy.io as sio
from tqdm import tqdm
import time
import importlib

import nnutils as utils

importlib.reload(utils)

<module 'nnutils' from '/home/jacobpri/git/RotspaceIT/notebooks/nnutils.py'>

## hyperparameters

In [352]:
arch = 'resnet18'
trained_on = 'object'
imageset = 'mc8-lummatched'

load_arch = arch + '-' + trained_on

## directory management

In [353]:
home_dir = '/home/jacobpri/git/RotspaceIT/'

# target directory for saving activations
activation_savedir = join(home_dir,'data','d02_modeling','activations')

# folder of trained model weights
weight_dir = join(home_dir,'data','d02_modeling','weights')

# folder of image sets
imageset_dir = join(home_dir,'imagesets')

dirs = os.listdir(imageset_dir)

found = False
for d in dirs:
    d_ = os.listdir(join(imageset_dir,d))
    if (imageset in d_):
        imageset_dir = join(imageset_dir, d, imageset)
        activation_savedir = join(activation_savedir, load_arch, d, imageset)
        found = True
        break

if found is False:
    raise ValueError('image set not found')   

# create savedir if it doesn't yet exist 
os.makedirs(activation_savedir, exist_ok=True)
print("imageset dir %s" % imageset_dir)
print("weight dir %s" % weight_dir)
print("activation save destination: %s" % activation_savedir)

assert(exists(activation_savedir))
assert(exists(imageset_dir))
assert(exists(weight_dir))

imageset dir /home/jacobpri/git/RotspaceIT/imagesets/experimental/mc8-lummatched
weight dir /home/jacobpri/git/RotspaceIT/data/d02_modeling/weights
activation save destination: /home/jacobpri/git/RotspaceIT/data/d02_modeling/activations/resnet18-object/experimental/mc8-lummatched


## load architecture

In [354]:
if load_arch == 'alexnet':
    model = models.alexnet(pretrained = True)
elif load_arch == 'alexnet-random':
    model = models.alexnet()
elif load_arch == 'alexnet-object':
    checkpoint = torch.load(os.path.join(weight_dir,'alexnet_imagenet_final.pth.tar'),map_location='cpu')
    checkpoint['state_dict'] = {str.replace(k,'module.',''): v for k,v in checkpoint['state_dict'].items()}
    model = models.alexnet(num_classes=1000)
    model.load_state_dict(checkpoint['state_dict'])
elif load_arch == 'alexnet-face':
    #model = models.__dict__['alexnet'](num_classes=9131)
    checkpoint = torch.load(os.path.join(weight_dir,'alexnet_faces_final.pth.tar'),map_location='cpu')
    #print(checkpoint['state_dict'].keys())
    checkpoint['state_dict'] = {str.replace(k,'module.',''): v for k,v in checkpoint['state_dict'].items()}
    model = models.alexnet(num_classes=3372)
    model.load_state_dict(checkpoint['state_dict'])
elif load_arch == 'alexnet-scene':
    checkpoint = torch.load(os.path.join(weight_dir,'alexnet_places_final.pth.tar'),map_location='cpu')
    checkpoint['state_dict'] = {str.replace(k,'module.',''): v for k,v in checkpoint['state_dict'].items()}
    model = models.alexnet(num_classes=365)
    model.load_state_dict(checkpoint['state_dict'])
elif load_arch == 'resnet18-object':
    model = models.resnet18(pretrained = True)
elif load_arch == 'vgg16-object':
    model = models.vgg16(pretrained = True)
else:
    raise ValueError('model name not recognized')

print(model)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

## get string names/indices of each layer 

In [355]:
lay_names_torch, lay_names_user, lay_names_user_fmt = utils.get_layer_names(model)

['1_conv1', '2_norm1', '3_relu1', '4_maxpool1', '5_conv2', '6_norm2', '7_relu2', '8_conv3', '9_norm3', '10_conv4', '11_norm4', '12_relu4', '13_conv5', '14_norm5', '15_conv6', '16_norm6', '17_relu6', '18_conv7', '19_norm7', '20_downsample7', '21_norm7', '22_conv8', '23_norm8', '24_relu8', '25_conv9', '26_norm9', '27_conv10', '28_norm10', '29_relu10', '30_conv11', '31_norm11', '32_downsample11', '33_norm11', '34_conv12', '35_norm12', '36_relu12', '37_conv13', '38_norm13', '39_conv14', '40_norm14', '41_relu14', '42_conv15', '43_norm15', '44_downsample15', '45_norm15', '46_conv16', '47_norm16', '48_relu16', '49_conv17', '50_norm17', '51_avgpool17', '52_fc18']


## convert model to instrumented model

In [356]:
# you only want to do this conversion ONCE
if not isinstance(model, nethook.InstrumentedModel):

    # convert to instrumented model
    model = nethook.InstrumentedModel(model)
else:
    print('warning: network is already InstrumentedModel')

## choose layers to retain (for extracting activations)

In [360]:
all_types = ['conv','relu','pool','norm','drop','fc','downsample']
includes = ['conv','relu','fc']#all_types#['conv','fc']

layers_to_retain = []
layers_to_retain_fmt = []

for i in range(len(lay_names_user_fmt)):
    if any(s in lay_names_user_fmt[i] for s in includes):
        layers_to_retain.append(lay_names_torch[i])
        layers_to_retain_fmt.append(lay_names_user_fmt[i])

print(layers_to_retain,'\n')
print(layers_to_retain_fmt,'\n')
print('%d layers chosen' % len(layers_to_retain))

['conv1', 'relu', 'layer1.0.conv1', 'layer1.0.relu', 'layer1.0.conv2', 'layer1.1.conv1', 'layer1.1.relu', 'layer1.1.conv2', 'layer2.0.conv1', 'layer2.0.relu', 'layer2.0.conv2', 'layer2.1.conv1', 'layer2.1.relu', 'layer2.1.conv2', 'layer3.0.conv1', 'layer3.0.relu', 'layer3.0.conv2', 'layer3.1.conv1', 'layer3.1.relu', 'layer3.1.conv2', 'layer4.0.conv1', 'layer4.0.relu', 'layer4.0.conv2', 'layer4.1.conv1', 'layer4.1.relu', 'layer4.1.conv2', 'fc'] 

['1_conv1', '3_relu1', '5_conv2', '7_relu2', '8_conv3', '10_conv4', '12_relu4', '13_conv5', '15_conv6', '17_relu6', '18_conv7', '22_conv8', '24_relu8', '25_conv9', '27_conv10', '29_relu10', '30_conv11', '34_conv12', '36_relu12', '37_conv13', '39_conv14', '41_relu14', '42_conv15', '46_conv16', '48_relu16', '49_conv17', '52_fc18'] 

27 layers chosen


## retain chosen layers of instrumented model

In [361]:
for layer_name in layers_to_retain: model.retain_layer(layer_name) 

In [ ]:
## 

In [260]:
model2 = models.alexnet()

In [261]:
list(model2.named_children())

[('features', Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )),
 ('avgpool', AdaptiveAvgPool2d(output_size=(6, 6))),
 ('classifier', Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
    (2

In [266]:
model_ = model2

lay_names_torch = []
lay_names_user = []
lay_conv_strs = []
layers = list(model_.named_modules())
count = 0
conv_count = 0

for i in range(0,len(layers)):
    #print(i)
    module_type = layers[i][1].__class__.__name__
    
    nonbasic_types = np.array(['Sequential','BasicBlock','Bottleneck','Fire',
                             '_DenseBlock', '_DenseLayer', 'Transition','InvertedResidual','_InvertedResidual','ConvBNReLU'])
    
    conv_types = ['conv','linear','fc']
    conv_types = ['Conv','Linear']
    
    if np.logical_not(np.isin(module_type, nonbasic_types)):
        
        # get layer naming info
        lay_names_torch.append(layers[i][0])
        lay_names_user.append(str(count) + '_' + str(layers[i][1]).split('(')[0])

        # get conv tag
        if any(s in lay_names_user[-1] for s in conv_types) and 'downsample' not in lay_names_torch[-1]:
            conv_count += 1
        lay_conv_strs.append(str(conv_count))
        
        # update
        lay_names_user[-1] = lay_names_user[-1] + '_' + lay_conv_strs[-1]

        count += 1
        

In [267]:
print(lay_names_torch)

['', 'features.0', 'features.1', 'features.2', 'features.3', 'features.4', 'features.5', 'features.6', 'features.7', 'features.8', 'features.9', 'features.10', 'features.11', 'features.12', 'avgpool', 'classifier.0', 'classifier.1', 'classifier.2', 'classifier.3', 'classifier.4', 'classifier.5', 'classifier.6']


In [268]:
for i in range(1,len(lay_names_torch)):
    #num_idx[i][0] += 1
    print((lay_names_torch[i],lay_names_user[i]))#,lay_conv_strs[i]))

#print(lay_types)
#print(len(lay_names))

('features.0', '1_Conv2d_1')
('features.1', '2_ReLU_1')
('features.2', '3_MaxPool2d_1')
('features.3', '4_Conv2d_2')
('features.4', '5_ReLU_2')
('features.5', '6_MaxPool2d_2')
('features.6', '7_Conv2d_3')
('features.7', '8_ReLU_3')
('features.8', '9_Conv2d_4')
('features.9', '10_ReLU_4')
('features.10', '11_Conv2d_5')
('features.11', '12_ReLU_5')
('features.12', '13_MaxPool2d_5')
('avgpool', '14_AdaptiveAvgPool2d_5')
('classifier.0', '15_Dropout_5')
('classifier.1', '16_Linear_6')
('classifier.2', '17_ReLU_6')
('classifier.3', '18_Dropout_6')
('classifier.4', '19_Linear_7')
('classifier.5', '20_ReLU_7')
('classifier.6', '21_Linear_8')


In [310]:
def get_layer_names(model):
    
    lay_names_torch = []
    lay_names_user = []
    lay_conv_strs = []
    
    layers = list(model.named_modules())
    
    count = 1
    conv_count = 0

    for i in range(1,len(layers)):
        
        module_type = layers[i][1].__class__.__name__

        nonbasic_types = np.array(['Sequential','BasicBlock','Bottleneck','Fire',
                                 '_DenseBlock', '_DenseLayer', 'Transition', '_Transition','InvertedResidual','_InvertedResidual','ConvBNReLU'])

        conv_types = ['Conv','Linear']

        if np.logical_not(np.isin(module_type, nonbasic_types)):

            # get layer naming info
            lay_names_torch.append(layers[i][0])
            lay_names_user.append(str(count) + '_' + str(layers[i][1]).split('(')[0])

            # get conv tag
            if any(s in lay_names_user[-1] for s in conv_types) and 'downsample' not in lay_names_torch[-1]:
                conv_count += 1
            lay_conv_strs.append(str(conv_count))

            # update
            lay_names_user[-1] = lay_names_user[-1] + '_' + lay_conv_strs[-1]

            count += 1
    
    lay_names_user_fmt = []
    
    for lay in lay_names_user:
        lay_type = lay.split('_')[1]
        
        if 'Conv' in lay_type:
            fmt = 'conv'
        elif 'Norm' in lay_type:
            fmt = 'norm'
        elif 'ReLU' in lay_type:
            fmt = 'relu'
        elif 'MaxPool' in lay_type:
            fmt = 'maxpool'
        elif 'AvgPool' in lay_type:
            fmt = 'avgpool'
        elif 'Linear' in lay_type:
            fmt = 'fc'
        elif 'Dropout' in lay_type:
            fmt = 'drop'
        else:
            print(lay_type)
            raise ValueError('fmt not implemented yet')
        
        lay_names_user_fmt.append(lay.split('_')[0] + '_' + fmt + lay.split('_')[2])

    return lay_names_torch, lay_names_user, lay_names_user_fmt


In [301]:
get_layer_strs(models.inception_v3())

KeyboardInterrupt: 

## convert to instrumented model

## retain all layers